# 1. Establaciendo conexión con el Sentinel 2

In [29]:
from sentinelhub import SHConfig
from dotenv import load_dotenv, find_dotenv
import os
import re
from imageio import imwrite
import rasterio

load_dotenv(find_dotenv(filename=".env"), override=True)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import numpy as np
from rasterio.transform import from_origin

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

config = SHConfig(  
    instance_id='',
    sh_client_id=CLIENT_ID,
    sh_client_secret=CLIENT_SECRET,
    sh_base_url='https://services.sentinel-hub.com',
    sh_auth_base_url='https://services.sentinel-hub.com',
   
)


## 2, 3 Descargando los .tiff de cada uno de los lagos en intervalo de tiempo

Puntos de interes

In [15]:
lagos = {
    "lago_atitlan" : {
        "west": -91.326256,
        "east": -91.07151,
        "south": 14.5948,
        "north": 14.750979
    },
    "lago_amatitlan": {
        "west": -90.638065,
        "east": -90.512924,
        "south": 14.412347,
        "north": 14.493799
    }
}

# 4, Deteccion de Cianobacteria

In [18]:
evalscript_cyano = """
// evalscript_both (RGB + chl)
//VERSION=3
var MNDWI_threshold=0.42, NDWI_threshold=0.4, filter_UABS=true, filter_SSI=false;

function setup() {
  return {
    input: [{ bands:["B02","B03","B04","B05","B07","B08","B8A","B11","B12"] }],
    output: [
      { id:"rgb", bands:3, sampleType:"AUTO" },
      { id:"chl", bands:1, sampleType:"FLOAT32" }
    ]
  };
}

function wbi(r,g,b,nir,swir1,swir2){
  let ws=0;
  try{
    var ndvi=(nir-r)/(nir+r), mndwi=(g-swir1)/(g+swir1), ndwi=(g-nir)/(g+nir),
        ndwi_leaves=(nir-swir1)/(nir+swir1),
        aweish=b+2.5*g-1.5*(nir+swir1)-0.25*swir2,
        aweinsh=4*(g-swir1)-(0.25*nir+2.75*swir1),
        dbsi=((swir1-g)/(swir1+g))-ndvi;
    if (mndwi>0.42||ndwi>0.4||aweinsh>0.1879||aweish>0.1112||ndvi<-0.2||ndwi_leaves>1) ws=1;
    if (filter_UABS && ws==1){ if (aweinsh<=-0.03 || dbsi>0) ws=0; }
  }catch(e){ws=0;}
  return ws;
}

function evaluatePixel(s){
  let water = wbi(s.B04,s.B03,s.B02,s.B08,s.B11,s.B12);
  let FAIv  = (s.B07 - s.B04 - (s.B8A - s.B04)*(783-665)/(865-665));
  let ndci  = (s.B05 - s.B04) / (s.B05 + s.B04);
  let chl   = 826.57*ndci**3 - 176.43*ndci**2 + 19*ndci + 4.071;

  let trueColor = [3*s.B04, 3*s.B03, 3*s.B02];
  let color;

  if (water==0) color = trueColor;
  else if (FAIv>0.08) color = [233/255,72/255,21/255];
  else if (chl<0.5)   color = [0,0,1.0];
  else if (chl<1)     color = [0,0,1.0];
  else if (chl<2.5)   color = [0,59/255,1];
  else if (chl<3.5)   color = [0,98/255,1];
  else if (chl<5)     color = [15/255,113/255,141/255];
  else if (chl<7)     color = [14/255,141/255,120/255];
  else if (chl<8)     color = [13/255,141/255,103/255];
  else if (chl<10)    color = [30/255,226/255,28/255];
  else if (chl<14)    color = [42/255,226/255,28/255];
  else if (chl<18)    color = [68/255,226/255,28/255];
  else if (chl<20)    color = [68/255,226/255,28/255];
  else if (chl<24)    color = [134/255,247/255,0];
  else if (chl<28)    color = [140/255,247/255,0];
  else if (chl<30)    color = [205/255,237/255,0];
  else if (chl<38)    color = [208/255,240/255,0];
  else if (chl<45)    color = [208/255,240/255,0];
  else if (chl<50)    color = [251/255,210/255,3/255];
  else if (chl<75)    color = [248/255,207/255,2/255];
  else if (chl<90)    color = [134/255,247/255,0];
  else if (chl<100)   color = [245/255,164/255,9/255];
  else if (chl<150)   color = [240/255,159/255,8/255];
  else if (chl<250)   color = [237/255,157/255,7/255];
  else if (chl<300)   color = [239/255,118/255,15/255];
  else if (chl<350)   color = [239/255,101/255,15/255];
  else if (chl<450)   color = [239/255,100/255,14/255];
  else if (chl<500)   color = [233/255,72/255,21/255];
  else                color = [233/255,72/255,21/255];

  // Devuelve imagen RGB y, en paralelo, el valor numérico (NaN fuera de agua)
  return {
    rgb: color,
    chl: [water ? chl : NaN]
  };
}
"""

Definiendo BBOX's

In [19]:
resolution = 60 # max 200

#Amatitlan
betsiboka_coords_wgs84_amatitlan = (
    lagos["lago_amatitlan"]["west"], 
    lagos["lago_amatitlan"]["south"], 
    lagos["lago_amatitlan"]["east"], 
    lagos["lago_amatitlan"]["north"]
) 

betsiboka_bbox_amatitlan = BBox(bbox=betsiboka_coords_wgs84_amatitlan, crs=CRS.WGS84)
betsiboka_size_amatitlan = bbox_to_dimensions(betsiboka_bbox_amatitlan, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size_amatitlan} pixels")


# Atitlan
betsiboka_coords_wgs84_atitlan = (
    lagos["lago_atitlan"]["west"],
    lagos["lago_atitlan"]["south"],
    lagos["lago_atitlan"]["east"],
    lagos["lago_atitlan"]["north"]
)

betsiboka_bbox_atitlan = BBox(bbox=betsiboka_coords_wgs84_atitlan, crs=CRS.WGS84)
betsiboka_size_atitlan = bbox_to_dimensions(betsiboka_bbox_atitlan, resolution=resolution)

# Esto serviera para iterar el request mas adelante :)
box_and_sizes = {
    "Amatitlan": {
        "bbox": betsiboka_bbox_amatitlan,
        "size": betsiboka_size_amatitlan
    },
    "atitlan": {
        "bbox": betsiboka_bbox_atitlan,
        "size": betsiboka_size_atitlan
    }
}

print(f"Image shape at {resolution} m resolution: {betsiboka_size_atitlan} pixels")

Image shape at 60 m resolution: (223, 153) pixels
Image shape at 60 m resolution: (455, 292) pixels



Ahora con los intervalos de tiempo, se descaragaran los .tiff para cada lago

In [27]:
start = datetime.datetime(2024, 6, 1)
end = datetime.datetime(2025, 6, 30)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

Monthly time windows:

('2024-06-01', '2024-07-01')
('2024-07-01', '2024-07-31')
('2024-07-31', '2024-08-30')
('2024-08-30', '2024-09-30')
('2024-09-30', '2024-10-30')
('2024-10-30', '2024-11-29')
('2024-11-29', '2024-12-30')
('2024-12-30', '2025-01-29')
('2025-01-29', '2025-02-28')
('2025-02-28', '2025-03-31')
('2025-03-31', '2025-04-30')
('2025-04-30', '2025-05-30')


In [31]:
start = datetime(2024, 6, 1)
end = datetime(2025, 6, 30)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Daily requests in each monthly window:\n")
for slot in slots:
    print(f"Interval: {slot[0]} to {slot[1]}")
    current_day = slot[0]
    while current_day < slot[1]:
        next_day = current_day + timedelta(days=1)
        for lake in box_and_sizes:
            # slug del nombre del lago (solo letras/números/guiones)
            lake_slug = re.sub(r'[^a-z0-9_-]+', '-', lake.lower())
            folder = f"out/{current_day.strftime('%Y-%m-%d')}_{lake_slug}"
            os.makedirs(folder, exist_ok=True)


            request_both = SentinelHubRequest(
                data_folder=folder,
                evalscript=evalscript_cyano,  # evalscript con outputs id="rgb" y id="chl"
                input_data=[
                    SentinelHubRequest.input_data(
                        data_collection=DataCollection.SENTINEL2_L1C,  # o SENTINEL2_L2A
                        time_interval=(current_day.isoformat(), next_day.isoformat()),
                        mosaicking_order=MosaickingOrder.LEAST_CC,
                    )
                ],
                responses=[
                    SentinelHubRequest.output_response("rgb", MimeType.PNG),   # visual
                    SentinelHubRequest.output_response("chl", MimeType.TIFF),  # numérico mg/m³
                ],
                bbox=box_and_sizes[lake]["bbox"],
                size=box_and_sizes[lake]["size"],
                config=config,
            )

            # Descarga y guarda en el folder con fecha_nombre
            out = request_both.get_data(save_data=False)
            if not out:
                print(f"[{lake} {current_day}] sin datos")
                continue
            
            rgb_img = out[0]["rgb.png"]
            chl_img = out[0]["chl.tif"] 
            
            # -------- Guardar RGB como PNG --------
            rgb_path = os.path.join(folder, "rgb.png")
            imwrite(rgb_path, rgb_img)
            
            # -------- Guardar CHL como GeoTIFF --------
            
            bbox = box_and_sizes[lake]["bbox"]
            height, width = chl_img.shape
            pixel_width  = (bbox.max_x - bbox.min_x) / width
            pixel_height = (bbox.max_y - bbox.min_y) / height
            transform = from_origin(bbox.min_x, bbox.max_y, pixel_width, pixel_height)
            
            crs = "EPSG:4326"

            chl_path = os.path.join(folder, "chl.tif")
            with rasterio.open(
                chl_path,
                "w",
                driver="GTiff",
                height=height,
                width=width,
                count=1,
                dtype=chl_img.dtype,
                crs=crs,
                transform=transform,
                nodata=np.nan,
                compress="deflate"
            ) as dst:
                dst.write(chl_img, 1)

            print(f"[{lake} {current_day}] guardado: {rgb_path} / {chl_path}")


        current_day = next_day

Daily requests in each monthly window:

Interval: 2024-06-01 to 2024-07-01
[Amatitlan 2024-06-01] guardado: out/2024-06-01_amatitlan\rgb.png / out/2024-06-01_amatitlan\chl.tif
[atitlan 2024-06-01] guardado: out/2024-06-01_atitlan\rgb.png / out/2024-06-01_atitlan\chl.tif
[Amatitlan 2024-06-02] guardado: out/2024-06-02_amatitlan\rgb.png / out/2024-06-02_amatitlan\chl.tif
[atitlan 2024-06-02] guardado: out/2024-06-02_atitlan\rgb.png / out/2024-06-02_atitlan\chl.tif
[Amatitlan 2024-06-03] guardado: out/2024-06-03_amatitlan\rgb.png / out/2024-06-03_amatitlan\chl.tif
[atitlan 2024-06-03] guardado: out/2024-06-03_atitlan\rgb.png / out/2024-06-03_atitlan\chl.tif
[Amatitlan 2024-06-04] guardado: out/2024-06-04_amatitlan\rgb.png / out/2024-06-04_amatitlan\chl.tif
[atitlan 2024-06-04] guardado: out/2024-06-04_atitlan\rgb.png / out/2024-06-04_atitlan\chl.tif
[Amatitlan 2024-06-05] guardado: out/2024-06-05_amatitlan\rgb.png / out/2024-06-05_amatitlan\chl.tif
[atitlan 2024-06-05] guardado: out/2024-